# Problem 1

## Since we have demand for each cost, the demand vector can be directly built. Then, we know that we can increase or decrease the production in each month with special stuff, where the increase and decrease action can be recorded by the variables. Next, we also know that the storage will add extra cost, which means the storage in each month must be recorded. 

## Then according to the problem we have, we need to reduce the cost in production, storage and demand not satisfied. All these values are set to be positive.

## Two sets of independent variables are used: increase of production in each month and decrease of production in each month.

## With these two sets of independent variables, the production in each month can be determined: $$p[t] = p[t-1] + incre[t] - decre[t]$$

## Then the storage in each month(assumed to be >=0), can be calculatedas:$$stor[t] = stor[t-1] + p[t] - d[t]$$

## Also the $stor[6]>=2$ is a required condition. In the last month, the storage is the stroage in last month plus the production in this month minus the unsatisfied demand before minus the demand in Dec. When the right side is smaller than 0, the stor[t] is 0.

## Similarly, the demand not satisfied can also be calcualted: $$inshort[t] == d[t] - stor[t-1] - p[t]$$ where the right side is smaller than 0, meaning the demand in this month is totally satisfied, the unsatisfied will automatically be set as 0. Also, the demand not satisfied in last month should be 0, in which we can have extra storage.

## Because the demand unsatisfied before should all be satisfied in Dec, we will have another constrain:$$sum(stor[5] + p[6] - inshort[1:5]) - d[6] >= 2)$$ meaning we will have all demand satisfied at the end of the year and at least 2 ton storage will be left.

## Another thing we need to remember is the storage cost. We see that the function of cost toward storage is a staged function. Here to realize the optimization we change it into following form:$$storc[t] == 60 + 75*stor[t]$$ where the cost function becomes convex one. In  this way optimize will trend to reduce the amount of storage.

## Finally the objective is minimize:$$240*sum(incre) + 210*sum(decre) + sum(storc) + 80*sum(inshort)$$ 

In [56]:
using JuMP, Clp

d = [10 5 8 20 15 3] # monthly bacteria demand

m = Model(Clp.Optimizer)

@variable(m, incre[1:6] >= 0 ) # increasing enzyme used in each month t=1,2,3,4,5,6
@variable(m, decre[1:6] >= 0 ) # decreasing enzyme used in each month
@variable(m, p[1:6] >= 0 ) # production in each month
@variable(m, stor[1:6] >= 0 ) # bacteria in storage
@variable(m, inshort[1:6] >= 0 ) # unmeet demand in each month
@variable(m, storc[1:6] >= 0) #storage cost in each month
# our objective is to minimize the total cost
@objective(m, Min, 240*sum(incre) + 210*sum(decre) + sum(storc) + 80*sum(inshort))

# constraint on the production in each month
@constraint(m, prod_rela_init, p[1] == 10 + incre[1] - decre[1])
@constraint(m, prod_rela[t in 2:6], p[t] == p[t-1] + incre[t] - decre[t])

# stroage in each month
@constraint(m, stor_rela_init, stor[1] == 1 + p[1] - d[1])
@constraint(m, stor_rela[t in 2:5], stor[t] == stor[t-1] + p[t] - d[t])
@constraint(m, stor_rela_Dec, stor[6] == stor[5] + p[6]-sum(inshort[1:5]) - d[6])

# demand not be satisfied
@constraint(m, inshort_rela_init, inshort[1] == d[1] - p[1] - 1)
@constraint(m, inshort_rela[t in 2:5], inshort[t] == d[t] - stor[t-1] - p[t])
@constraint(m, inshort_rela_Dec, inshort[6] == 0)

# unsatisfied demand should be satisfied in Dec
@constraint(m, inshort_sat, stor[5] + p[6] - sum(inshort[1:5]) - d[6] >= 2)

# 
@constraint(m, storc_rela[t in 1:6], storc[t] == 60 + 75*stor[t])
# solve this instance of ShoeCo and print relevant solution details
optimize!(m)
# pint the results
println("Increse ", Array(value.(incre)), " tons of bacteria in each month")
println("Decrease ", Array(value.(decre)), " tons of bacteria in each month")
println("Produce ", Array(value.(p')), " tons of bacteria in each month")
println("Storage: ", Array(value.(stor')))
println("Demand not satisfied: ", Array(value.(inshort')))
println("Cost: ", objective_value(m))

Increse [0.0, 0.0, 3.000000000000001, 12.0, 0.0, 0.0] tons of bacteria in each month
Decrease [1.0, 4.000000000000001, 0.0, 0.0, 5.0, 0.0] tons of bacteria in each month
Produce [9.0 4.999999999999999 8.0 20.0 15.0 15.0] tons of bacteria in each month
Storage: [0.0 0.0 0.0 0.0 0.0 11.999999999999998]
Demand not satisfied: [0.0 0.0 0.0 0.0 0.0 0.0]
Cost: 6960.0
Coin0506I Presolve 12 (-13) rows, 20 (-16) columns and 47 (-32) elements
Clp0006I 0  Obj 6140.8996 Primal inf 10.998657 (11)
Clp0006I 11  Obj 6960
Clp0000I Optimal - objective value 6960
Coin0511I After Postsolve, objective 6960, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 6960 - 11 iterations time 0.002, Presolve 0.00


## Generally, this optimal result is still ok. Where we can see only demand in one month is not satisfied totally. And the increase of producition is only need in the month has larhest demand. The increase and decrease of production will not appear in same month. At the end of the year we will still have enough storage. In this model, we will pay more to decrease the prodcution in last month than store the extra producted bacteria. So we will still have 12 storage in last month.

# Problem 2

## Here forgive me using nonlinear modulus first. In my evaluation, the cost of transfer is determiend by the modes on two neighbor legs:$$x_{i,leg1}*A_{i,j}*x_{j, leg2}$$where we will see quadratic terms of unknown variables.

In [91]:
using JuMP, Ipopt, NamedArrays
# create name of legs and modes
legs  = [1, 2, 3, 4]
modes = [:Tunnel, :Aircraft, :Teleportation]

Transcost = NamedArray( [0 500 1200; 800 0 1000; 1500 1000 0], (modes,modes), ("From","To"))

Infracost = NamedArray( [30 25 40 60; 25 40 45 50; 40 20 50 45], (modes,legs), ("Modes","Legs"))

m = Model(Ipopt.Optimizer)

@variable(m, x[modes,legs] >= 0) # x[i,j] is the choose of transport modes

@constraint(m, onemode[j in legs], sum(x[i,j] for i in modes) == 1)

@objective(m, Max, sum(25*x[i,j]*Infracost[i,j] for i in modes, j in legs) + 
    sum(x[i,leg[j]]*Transcost[i,k]*x[k,leg[j+1]] for i in modes, k in modes, j in 1:3))

500 x[Aircraft,2]*x[Tunnel,1] + 500 x[Aircraft,3]*x[Tunnel,2] + 500 x[Aircraft,4]*x[Tunnel,3] + 1200 x[Teleportation,2]*x[Tunnel,1] + 1200 x[Teleportation,3]*x[Tunnel,2] + 1200 x[Teleportation,4]*x[Tunnel,3] + 800 x[Tunnel,2]*x[Aircraft,1] + 800 x[Tunnel,3]*x[Aircraft,2] + 800 x[Tunnel,4]*x[Aircraft,3] + 1000 x[Teleportation,2]*x[Aircraft,1] + 1000 x[Teleportation,3]*x[Aircraft,2] + 1000 x[Teleportation,4]*x[Aircraft,3] + 1500 x[Tunnel,2]*x[Teleportation,1] + 1500 x[Tunnel,3]*x[Teleportation,2] + 1500 x[Tunnel,4]*x[Teleportation,3] + 1000 x[Aircraft,2]*x[Teleportation,1] + 1000 x[Aircraft,3]*x[Teleportation,2] + 1000 x[Aircraft,4]*x[Teleportation,3] + 750 x[Tunnel,1] + 625 x[Tunnel,2] + 1000 x[Tunnel,3] + 1500 x[Tunnel,4] + 625 x[Aircraft,1] + 1000 x[Aircraft,2] + 1125 x[Aircraft,3] + 1250 x[Aircraft,4] + 1000 x[Teleportation,1] + 500 x[Teleportation,2] + 1250 x[Teleportation,3] + 1125 x[Teleportation,4]

In [107]:
using JuMP
optimize!(m)

##solution = NamedArray(Int[value(x[i,j]) for i in modes, j in legs], (modes,legs), ("Modes","Legs"))
println("The construction plan is:")
println(NamedArray(value.(x[i,j] for i in modes, j in legs), (modes,legs), ("Modes","Legs")))
println("Total cost will be \$", objective_value(m))

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       12
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       18

Total number of variables............................:       12
                     variables with only lower bounds:       12
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        4
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

## I discussed with other students and found another form, even I don't think it is ok (I think its physical meaning is not very clear) to express the cost function in this way, I still pose this form here:

In [112]:
using JuMP, Clp, NamedArrays
# create name of legs and modes
legs  = [1, 2, 3, 4]
modes = [:Tunnel, :Aircraft, :Teleportation]
leg  = [2, 3, 4]
mode = [:Aircraft, :Teleportation, :Tunnel]
Transcost = NamedArray( [0 500 1200; 800 0 1000; 1500 1000 0], (modes,modes), ("From","To"))

Infracost = NamedArray( [30 25 40 60; 25 40 45 50; 40 20 50 45], (modes,legs), ("Modes","Legs"))

m = Model(Clp.Optimizer)

@variable(m, x[modes,legs] >= 0) # x[i,j] is the choose of transport modes

@constraint(m, onemode[j in legs], sum(x[i,j] for i in modes) == 1)

@objective(m, Max, sum(25*x[i,j]*Infracost[i,j] + x[i,j]*Transcost[i,k] for j in legs, (i,k) in zip(modes,mode)))

optimize!(m)

##solution = NamedArray(Int[value(x[i,j]) for i in modes, j in legs], (modes,legs), ("Modes","Legs"))
println("The construction plan is:")
println(NamedArray(value.(x[i,j] for i in modes, j in legs), (modes,legs), ("Modes","Legs")))
println("Total cost will be \$", objective_value(m))

The construction plan is:
3×4 Named Array{Float64,2}
  Modes ╲ Legs │   1    2    3    4
───────────────┼───────────────────
:Tunnel        │ 0.0  0.0  0.0  0.0
:Aircraft      │ 0.0  1.0  0.0  0.0
:Teleportation │ 1.0  0.0  1.0  1.0
Total cost will be $9875.0
Coin0506I Presolve 0 (-4) rows, 0 (-12) columns and 0 (-12) elements
Clp3002W Empty problem - 0 rows, 0 columns and 0 elements
Clp0000I Optimal - objective value 9875
Coin0511I After Postsolve, objective 9875, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 9875 - 0 iterations time 0.002, Presolve 0.00


## Generally, this method gives us a planning of infrastructure construction. But compared with optimal results based on nonlinear solver, I think the cost function here is not so physically meaningful.

## Here are some variables I used in the model

In [63]:
Infracost

3×4 Named Array{Int64,2}
  Modes ╲ Legs │  1   2   3   4
───────────────┼───────────────
:Tunnel        │ 30  25  40  60
:Aircraft      │ 25  40  45  50
:Teleportation │ 40  20  50  45

In [64]:
x

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, [:Tunnel, :Aircraft, :Teleportation]
    Dimension 2, [1, 2, 3, 4]
And data, a 3×4 Array{VariableRef,2}:
 x[Tunnel,1]         x[Tunnel,2]         …  x[Tunnel,4]
 x[Aircraft,1]       x[Aircraft,2]          x[Aircraft,4]
 x[Teleportation,1]  x[Teleportation,2]     x[Teleportation,4]

In [66]:
Transcost

3×3 Named Array{Int64,2}
     From ╲ To │        :Tunnel       :Aircraft  :Teleportation
───────────────┼───────────────────────────────────────────────
:Tunnel        │              0             500            1200
:Aircraft      │            800               0            1000
:Teleportation │           1500            1000               0

In [73]:
x[:Tunnel, leg[1]]

x[Tunnel,1]

In [76]:
for j in 1:1
    for i in modes
        for k in modes
        print(x[i,leg[1]]*Transcost[i,k]*x[k,leg[1+1]],'+')
        end
    end
end

0+500 x[Tunnel,1]*x[Aircraft,2]+1200 x[Tunnel,1]*x[Teleportation,2]+800 x[Aircraft,1]*x[Tunnel,2]+0+1000 x[Aircraft,1]*x[Teleportation,2]+1500 x[Teleportation,1]*x[Tunnel,2]+1000 x[Teleportation,1]*x[Aircraft,2]+0+

# Problem 3

## a) This problem is a typical assigning problem: asssigning 8 astronauts into 4 missions with the skill requirements met. So an optimal assignment of can be obtained with a model built and optimization applied.

## b) Since I answered 'Yes'. I tried to give a assignment:
## mission1: Astronaut 1 and 7
## mission2: Astronaut 2 and 8
## mission3: Astronaut 3 and 6
## mission4: Astronaut 4 and 5

## c) the skills in wich at least one astronaut has a score of ≥ 10/20:
## mission1: Computing/IT, Piloting, Mechanical engineering; Agriculture, Astrobiology, Electronics, Air & water
## mission2: Computing/IT, Mechanical engineering, Health/safety; Agriculture, Astrobiology, Electronics, Geology
## mission3: Mechanical engineering, Piloting; Agriculture, Air & water, Astrobiology, Geology
## mission4: Computing/IT, Piloting, Piloting, Health/safety; Electronics, Astrobiology, Electronics, Geology;

## To my comprehension we need to obtain a assignment of astronaut into 4 misssions, meaning we need a $x[i,j]$, where i in astronaut, j in missions

In [120]:
using JuMP, Clp, NamedArrays
# create
Astronaut = [:A1, :A2, :A3, :A4, :A5, :A6, :A7, :A8]
Skill = [:ComputingIT, :Piloting, :Mechanicalengineering, :Healthsafety, :Agriculture, :Electronics, :Astrobiology, :Airwater, :Geology]
Mission = [:M1, :M2, :M3, :M4]
Skill_astronaut = NamedArray( [20 14 0 13 0 0 8 8; 12 0 0 10 15 20 8 9; 0 20 12 0 8 11 14 12; 0 0 0 0 17 0 0 16;
        18 12 15 0 0 0 8 0; 10 0 9 14 15 8 12 13; 0 17 0 11 13 10 0 0; 0 0 14 0 0 12 16 0; 0 0 0 0 12 18 0 18],
    (Skill, Astronaut), ("Skill","Astronaut"))

9×8 Named Array{Int64,2}
     Skill ╲ Astronaut │ :A1  :A2  :A3  :A4  :A5  :A6  :A7  :A8
───────────────────────┼───────────────────────────────────────
:ComputingIT           │  20   14    0   13    0    0    8    8
:Piloting              │  12    0    0   10   15   20    8    9
:Mechanicalengineering │   0   20   12    0    8   11   14   12
:Healthsafety          │   0    0    0    0   17    0    0   16
:Agriculture           │  18   12   15    0    0    0    8    0
:Electronics           │  10    0    9   14   15    8   12   13
:Astrobiology          │   0   17    0   11   13   10    0    0
:Airwater              │   0    0   14    0    0   12   16    0
:Geology               │   0    0    0    0   12   18    0   18

In [130]:
m = Model(Clp.Optimizer)

@variable(m, x[Astronaut,Mission] >= 0) # x[i,j] is the choose of transport modes

@constraint(m, crew_two[j in Mission], sum(x[i,j] for i in Astronaut) == 2)

@constraint(m, astronaut_one[i in Astronaut], sum(x[i,j] for j in Mission) == 1)

@objective(m, Max, sum(Skill_astronaut[i,j]*x[j,k] for i in Skill, j in Astronaut, k in Mission))

optimize!(m)

println("The mission assignment is:")
println(NamedArray(value.(x[i,j] for i in Astronaut, j in Mission), (Astronaut,Mission), ("Astronaut","Mission")))
println("Maximum total score will be ", objective_value(m))

The mission assignment is:
8×4 Named Array{Float64,2}
Astronaut ╲ Mission │ :M1  :M2  :M3  :M4
────────────────────┼───────────────────
:A1                 │ 0.0  0.0  0.0  1.0
:A2                 │ 0.0  1.0  0.0  0.0
:A3                 │ 1.0  0.0  0.0  0.0
:A4                 │ 1.0  0.0  0.0  0.0
:A5                 │ 0.0  0.0  1.0  0.0
:A6                 │ 0.0  0.0  1.0  0.0
:A7                 │ 0.0  0.0  0.0  1.0
:A8                 │ 0.0  1.0  0.0  0.0
Maximum total score will be 522.0
Coin0506I Presolve 12 (0) rows, 32 (0) columns and 64 (0) elements
Clp0006I 0  Obj 0 Primal inf 15.999999 (12) Dual inf 2088 (32)
Clp0006I 18  Obj 522
Clp0000I Optimal - objective value 522
Clp0032I Optimal objective 522 - 18 iterations time 0.002


## Here every mission has two astronauts to do. And every astronaut is arranged into only one mission. In all, this result is acceptable in general.

# Problem 4

## Generally, this is a point to point transport problem. We can assume the transport from districts to a hub as decision variables:$x1[i,j]$, $x2[i,j]$ for hub1 and hub2, where i in district, j in equipment.

## Each transportation hub must have between 350 and 500 units of safety equipment each night, means:$$350\leq\sum_{i=0}^{3}{{x}_{ij}}\leq500$$

## The ratio of oxygen tanks in all equipment is 49% and then:$$44\%\leq\frac{\sum_{i=District}^{}{x_{1,ij}(j=Oxygen)}}{\sum_{i=District}^{}\sum_{i=Equip}^{}{x_{1,ij}}}\leq54\%$$

In [161]:
using JuMP, Clp, NamedArrays
# create
Equip = [:Oxygen, :Pressuresuits]
District = [:Finanical, :Residential, :Commerical]
Hub = [:Hub1, :Hub2]

District_equip = NamedArray( [45 210; 150 250; 120 190], (District, Equip), ("District","Equip"))

District_hub = NamedArray( [1.0 2.3; 2.1 0.9; 1.5 1.1], (District, Hub), ("District","Hub"))

m = Model(Clp.Optimizer)

@variable(m, x1[District,Equip] >= 0) # x1[i,j] is the transport of equipment from a district to hub1
@variable(m, x2[District,Equip] >= 0) # x2[i,j] is the transport of equipment from a district to hub2

@constraint(m, hub1_amount, 350 <= sum(x1[i,j] for i in District, j in Equip) <= 500)
@constraint(m, hub2_amount, 350 <= sum(x2[i,j] for i in District, j in Equip) <= 500)

@constraint(m, hub1_percent1, sum(x1[i,:Oxygen] for i in District) <=  0.54*sum(x1[i,j] for i in District, j in Equip))

@constraint(m, hub1_percent2, 0.44*sum(x1[i,j] for i in District, j in Equip) <= sum(x1[i,:Oxygen] for i in District))

@constraint(m, hub2_percent1, sum(x2[i,:Oxygen] for i in District) <=  0.54*sum(x2[i,j] for i in District, j in Equip))

@constraint(m, hub2_percent2, 0.44*sum(x2[i,j] for i in District, j in Equip) <= sum(x2[i,:Oxygen] for i in District))

@objective(m, Min, sum(x1[i,j]*District_hub[i,:Hub1] + x2[i,j]*District_hub[i,:Hub2] for i in District, j in Equip))

optimize!(m)

println("The transport plan to hub 1 is:")
println(NamedArray(value.(x1[i,j] for i in District, j in Equip), (District,Equip), ("District", "Equip")))
println("The transport plan to hub 2 is:")
println(NamedArray(value.(x2[i,j] for i in District, j in Equip), (District,Equip), ("District", "Equip")))
println("Minimum total distance will be ", objective_value(m))

The transport plan to hub 1 is:
3×2 Named Array{Float64,2}
District ╲ Equip │        :Oxygen  :Pressuresuits
─────────────────┼───────────────────────────────
:Finanical       │          189.0           161.0
:Residential     │            0.0             0.0
:Commerical      │            0.0             0.0
The transport plan to hub 2 is:
3×2 Named Array{Float64,2}
District ╲ Equip │        :Oxygen  :Pressuresuits
─────────────────┼───────────────────────────────
:Finanical       │            0.0             0.0
:Residential     │          189.0           161.0
:Commerical      │            0.0             0.0
Minimum total distance will be 665.0
Coin0506I Presolve 6 (0) rows, 4 (-8) columns and 12 (-24) elements
Clp0006I 0  Obj 0 Primal inf 700 (2)
Clp0006I 4  Obj 665
Clp0000I Optimal - objective value 665
Coin0511I After Postsolve, objective 665, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 665 - 4 iterations time 0.002, Presolve 0.00


## This result has a flaw that no equipment will be transformed from Commerical to hubs. Even the total distance is minimized, this means the equipment in Commercial cannot be updated in time.